In [1]:
#Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv
/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Daily_Ridership_Data__Beginning_2020.csv
/kaggle/input/mta-daily-ridership-data-beginning-2020/2023 Subway Tables - Annual Total.csv
/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload


# *Data Cleaning*

* Monthly Ridership Since 2018

In [2]:
#Importing data
monthly_riders_since_2018 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv')

#Filter to only include Subway and no other MTA agency
monthly_riders_since_2018 = monthly_riders_since_2018[monthly_riders_since_2018['Agency'] == 'Subway'] 

#Rename "Month" column into Date
monthly_riders_since_2018.rename(columns = {'Month':'Date'}, inplace = True)

#Polish Date column and convert to datetime
monthly_riders_since_2018['Date'] = monthly_riders_since_2018['Date'].str.replace('/01/', '/', regex=True)
monthly_riders_since_2018['Date'] = pd.to_datetime(monthly_riders_since_2018['Date'], format='%m/%Y')

#Break out date into columns for month and year
monthly_riders_since_2018['Month'] = monthly_riders_since_2018['Date'].dt.month
monthly_riders_since_2018['Year'] = monthly_riders_since_2018['Date'].dt.year

#Drop Agency column
monthly_riders_since_2018.drop(['Agency'], inplace=True, axis=1)

#Filter out the year 2024 to only have data from 2018 to 2023
monthly_riders_since_2018 = monthly_riders_since_2018[monthly_riders_since_2018['Date'] < '2024-02-01'] 

#Rearrange columns
monthly_riders_since_2018 = monthly_riders_since_2018.reindex(['Date', 'Month', 'Year', 'Ridership'], axis=1)


#################################
monthly_riders_since_2018

,Date,Month,Year,Ridership
238,2018-01-01,1,2018,134683435
242,2018-02-01,2,2018,127432835
246,2018-03-01,3,2018,143982923
251,2018-04-01,4,2018,141950369
255,2018-05-01,5,2018,150320833
...,...,...,...,...
679,2023-09-01,9,2023,95253595
686,2023-10-01,10,2023,103949018
693,2023-11-01,11,2023,98511566
700,2023-12-01,12,2023,95685578


<br>

* Daily Ridership Since 2020

In [3]:
#Importing data
daily_riders_since_2020 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Daily_Ridership_Data__Beginning_2020.csv', parse_dates=['Date'])

#No missing values
daily_riders_since_2020.dropna(axis=1, inplace=True)

#Drop unccesary columns that belong to other agencies outside of the subway 
daily_riders_since_2020.drop(daily_riders_since_2020.iloc[:, 3:], inplace=True, axis=1)

#Break out date into columns
daily_riders_since_2020['Day'] = daily_riders_since_2020['Date'].dt.day
daily_riders_since_2020['Month'] = daily_riders_since_2020['Date'].dt.month
daily_riders_since_2020['Year'] = daily_riders_since_2020['Date'].dt.year
daily_riders_since_2020['Day of Week'] = daily_riders_since_2020['Date'].dt.dayofweek

#Filter out the year 2024
daily_riders_since_2020 = daily_riders_since_2020[daily_riders_since_2020['Year'] < 2024] 

#Sort by date
daily_riders_since_2020.sort_values('Date', ascending=True, inplace=True)

#Renaming columns
daily_riders_since_2020.rename(columns={'Subways: Total Estimated Ridership':'Total Estimated Ridership', 
                                        'Subways: % of Comparable Pre-Pandemic Day': '% Comparable to Pre-Pandemic Days'}, 
                               inplace=True)

#Rearrange columns
daily_riders_since_2020 = daily_riders_since_2020.reindex(['Date', 'Day', 'Month', 'Year', 'Day of Week', 'Total Estimated Ridership', '% Comparable to Pre-Pandemic Days'], axis=1)


#################################
daily_riders_since_2020

,Date,Day,Month,Year,Day of Week,Total Estimated Ridership,% Comparable to Pre-Pandemic Days
1543,2020-03-01,1,3,2020,6,2212965,0.97
1542,2020-03-02,2,3,2020,0,5329915,0.96
1541,2020-03-03,3,3,2020,1,5481103,0.98
1540,2020-03-04,4,3,2020,2,5498809,0.99
1539,2020-03-05,5,3,2020,3,5496453,0.99
...,...,...,...,...,...,...,...
148,2023-12-27,27,12,2023,2,2912007,0.55
147,2023-12-28,28,12,2023,3,3064841,0.57
146,2023-12-29,29,12,2023,4,3198885,0.60
145,2023-12-30,30,12,2023,5,2440211,0.74


<br>

* Hourly Ridership in 2023

In [4]:
#Importing data
hourly_ridership_2023 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload', low_memory=False)

#Dropping missing values
hourly_ridership_2023.dropna(inplace=True)

#Convert timestamp column to Datetime
hourly_ridership_2023['transit_timestamp'] = pd.to_datetime(hourly_ridership_2023['transit_timestamp'], format='%m/%d/%Y %I:%M:%S %p')

#Breakout timestamp info into diffeent columns
hourly_ridership_2023['day'] = hourly_ridership_2023['transit_timestamp'].dt.day
hourly_ridership_2023['month'] = hourly_ridership_2023['transit_timestamp'].dt.month
hourly_ridership_2023['year'] = hourly_ridership_2023['transit_timestamp'].dt.year
hourly_ridership_2023['day_of_week'] = hourly_ridership_2023['transit_timestamp'].dt.dayofweek

#Query to further filter data
hourly_ridership_2023 = hourly_ridership_2023.query('borough != "Staten Island" and transit_mode != "tram" and station_complex_id != "TRAM1" and transit_timestamp > "2022-12-31 23:59:59" and transit_timestamp < "2024-01-01 00:00:00"')

#Convert Station ID to int
hourly_ridership_2023['station_complex_id'] = hourly_ridership_2023['station_complex_id'].astype(int)

#Drop unccesary columns for our analysis
hourly_ridership_2023.drop(columns=['payment_method', 'fare_class_category', 'transfers', 'Georeference', 'transit_mode'], inplace=True, axis=1)    

#Rearrange columns (Remember to Add above columns if you want to show Mean and Total Amounts)
hourly_ridership_2023 = hourly_ridership_2023.reindex(['transit_timestamp', 
                                                       'month', 
                                                       'day', 
                                                       'year', 
                                                       'day_of_week', 
                                                       'station_complex_id', 
                                                       'station_complex', 
                                                       'borough', 
                                                       'ridership', 
                                                       'latitude', 'longitude'], 
                                                      axis=1)

#Replacing commas 
hourly_ridership_2023['station_complex'] = hourly_ridership_2023['station_complex'].str.replace(',', '/')

#Cleaning up decimal places in latitude and longitude columns
hourly_ridership_2023['latitude'] = hourly_ridership_2023['latitude'].round(3)
hourly_ridership_2023['longitude'] = hourly_ridership_2023['longitude'].round(3)

    
#################################    
hourly_ridership_2023

,transit_timestamp,month,day,year,day_of_week,station_complex_id,station_complex,borough,ridership,latitude,longitude
0,2023-02-12 18:00:00,2,12,2023,6,164,34 St-Penn Station (A/C/E),Manhattan,84.0,40.752,-73.993
1,2023-03-28 14:00:00,3,28,2023,1,164,34 St-Penn Station (A/C/E),Manhattan,123.0,40.752,-73.993
2,2023-01-11 02:00:00,1,11,2023,2,164,34 St-Penn Station (A/C/E),Manhattan,3.0,40.752,-73.993
3,2023-06-08 17:00:00,6,8,2023,3,164,34 St-Penn Station (A/C/E),Manhattan,200.0,40.752,-73.993
4,2023-07-25 16:00:00,7,25,2023,1,164,34 St-Penn Station (A/C/E),Manhattan,872.0,40.752,-73.993
...,...,...,...,...,...,...,...,...,...,...,...
45109797,2023-08-19 06:00:00,8,19,2023,5,607,34 St-Herald Sq (B/D/F/M/N/Q/R/W),Manhattan,1.0,40.750,-73.988
45109798,2023-10-06 10:00:00,10,6,2023,4,607,34 St-Herald Sq (B/D/F/M/N/Q/R/W),Manhattan,5.0,40.750,-73.988
45109799,2023-06-01 16:00:00,6,1,2023,3,607,34 St-Herald Sq (B/D/F/M/N/Q/R/W),Manhattan,11.0,40.750,-73.988
45109800,2023-05-28 14:00:00,5,28,2023,6,607,34 St-Herald Sq (B/D/F/M/N/Q/R/W),Manhattan,1662.0,40.750,-73.988


<br>

* Total Ridership Since 2018

In [5]:
#Importing Data
total_ridership_since_2018 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/2023 Subway Tables - Annual Total.csv')

#Rework data header and columns
total_ridership_since_2018.columns = total_ridership_since_2018.iloc[0]
total_ridership_since_2018 = total_ridership_since_2018[1:]

#Renaming some columns
total_ridership_since_2018.rename(columns={'Station (alphabetical by borough)': 'station name',
                                           'Boro': 'borough',
                                           '2018': 'total_ridership_2018',
                                           '2019': 'total_ridership_2019',
                                           '2020': 'total_ridership_2020',
                                           '2021': 'total_ridership_2021',
                                           '2022': 'total_ridership_2022',
                                           '2023': 'total_ridership_2023',
                                           '2022-2023 Change': '2022-2023_change',
                                           '2023 Rank': '2023_ridership_rank'},
                                  inplace=True)

#Drop columns that are not needed
total_ridership_since_2018 = total_ridership_since_2018.iloc[:, :1].join(total_ridership_since_2018.iloc[:, 2:])
total_ridership_since_2018 = total_ridership_since_2018.iloc[:, :9].join(total_ridership_since_2018.iloc[:, 10:])

#Delete rows that don't have any numeric values for analysis
total_ridership_since_2018.dropna(subset=['borough'], inplace=True)

#Replace names of borough
replacements = {'Bx': 'The Bronx', 
                'Q': 'Queens', 
                'B': 'Brooklyn',
                'M': 'Manhattan'}
total_ridership_since_2018['borough'] = total_ridership_since_2018['borough'].replace(replacements)
        
#Change data type of numeric columns
columns = ['total_ridership_2018', 
           'total_ridership_2019', 
           'total_ridership_2020', 
           'total_ridership_2021', 
           'total_ridership_2022', 
           'total_ridership_2023', 
           '2022-2023_change', 
           '2023_ridership_rank']
for i in columns:
    total_ridership_since_2018[i] = total_ridership_since_2018[i].str.replace(',', '').astype(int)

    
#################################    
total_ridership_since_2018

,station name,borough,total_ridership_2018,total_ridership_2019,total_ridership_2020,total_ridership_2021,total_ridership_2022,total_ridership_2023,2022-2023_change,2023_ridership_rank
2,"138 St-Grand Concourse (4,5)",The Bronx,944598,1035878,371408,656866,766610,785271,18661,336
3,"149 St-Grand Concourse (2,4,5)",The Bronx,3972763,3931908,1815785,1832521,2026363,2087779,61416,154
4,"161 St-Yankee Stadium (B,D,4)",The Bronx,8392290,8254928,3221651,4077604,5023193,5316351,293158,54
5,167 St (4),The Bronx,2933140,2653237,1396287,1615072,1847368,1901393,54025,175
6,"167 St (B,D)",The Bronx,2022919,2734530,1422149,1508270,1492833,1411144,-81689,206
...,...,...,...,...,...,...,...,...,...,...
423,"Sutphin Blvd-Archer Av-JFK Airport (E,J,Z)",Queens,7282128,7354064,2951061,3799692,4770175,5941974,1171799,35
424,Vernon Blvd-Jackson Av (7),Queens,4493326,4623070,1768317,2189038,3113214,3631659,518445,81
425,"Woodhaven Blvd (J,Z)",Queens,1370889,1337787,681796,753419,838374,557522,-280852,361
426,"Woodhaven Blvd (M,R)",Queens,6509386,6381132,2634483,3302348,3905277,4237180,331903,66


In [6]:
monthly_riders_since_2018.to_csv('/kaggle/working/monthly_riders_since_2008.csv', index=False)

daily_riders_since_2020.to_csv('/kaggle/working/daily_riders_since_2020.csv', index=False)

hourly_ridership_2023.to_csv('/kaggle/working/hourly_ridership_since_2023.csv', index=False)

total_ridership_since_2018.to_csv('/kaggle/working/total_ridership_since_2018.csv', index=False)